In [ ]:
# default_exp uniformcf

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# todos


- CF Statement hat noch mehrere Qrts drin. prüfen, ob alle qtrs=1 haben.


# uniform CF

## Basic Settings

In [ ]:
# imports
from bfh_mt_hs2020_sec_data.core import * 
from pathlib import Path
from typing import List, Tuple, Union, Set
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import col

import pandas as pd

import shutil          # provides high level file operations
import time            # used to measure execution time
import os
import sys

In [ ]:
all_pivot_selected_folder  = "D:/data/parq_pivot_select"
all_pivoted_folder = "D:/data/parq_pivot_split"
all_processed_folder = "D:/data/parq_processed/"

col_list =    ["stmt","cik","ticker", "adsh","period","filed", "form","tag","value","report", "line", "fp", "uom"]
pivot_group = ["cik","ticker","adsh","form","period","filed","fp", "qtrs"]

In [ ]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

## 00_Raw_data

In [ ]:
# loading the complete unpivoted dataset - if it is needed for debbuging
df_all_selected = spark.read.parquet(all_pivot_selected_folder).cache()

In [ ]:
# it happens sometimes, that the data could not be associated with a right sheet (bs, is, cf, ..). in this cases, the data can appea under "UN"
# so if expected information cannot be found in the appropriate statement, we have to look in the un statement
un_pivot_value = load_data("UN", "value")
un_pivot_pd = un_pivot_value.toPandas()

In [ ]:
un_pivot_pd.shape

(5989, 1774)

In [ ]:
def prepare_un_values(df_to_merge_into, attr_list):
    # add possible columns from un set to  cf data with prefix cpy_
    attributes = pivot_group[:] # create copy
    attributes.extend(attr_list)

    un_prepared = un_pivot_pd[attributes].copy()
    un_prepared.rename(columns=lambda x: x  if x in pivot_group else ("cpy_" + x), inplace=True)

    return pd.merge(df_to_merge_into, un_prepared, how='left', on=pivot_group)

## 01_CashFlow

Operation
- NetIncomeLoss
- ProfitLoss
- NetCashProvidedByUsedInOperatingActivities: NetIncome + other positions ergibt diese Position

Investing
- NetCashProvidedByUsedInInvestingActivities

Financing activities
- PaymentsForRepurchaseOfCommonStock: Aktienrückkäufe
- PaymentsOfDividends
- NetCashProvidedByUsedInFinancingActivities

Cash Bestand unterschied
- CashAndCashEquivalentsPeriodIncreaseDecrease: increase/decrease in cash

In [ ]:
cf_pivot_value = load_data("CF", "value")
spark_shape(cf_pivot_value)

(305903, 2909)

In [ ]:
#cf_empty_count = get_empty_count(cf_pivot_value)

In [ ]:
cf_pivot_pd = cf_pivot_value.toPandas()

In [ ]:
cf_pivot_pd_copy = cf_pivot_pd.copy()

In [ ]:
cf_pivot_pd.shape

(305903, 2909)

## 02_CashFlow

Operation
- NetIncomeLoss
- ProfitLoss
- NetCashProvidedByUsedInOperatingActivities: NetIncome + other positions ergibt diese Position

Investing
- NetCashProvidedByUsedInInvestingActivities

Financing activities
- PaymentsForRepurchaseOfCommonStock: Aktienrückkäufe
- PaymentsOfDividends
- NetCashProvidedByUsedInFinancingActivities

Cash Bestand unterschied
- CashAndCashEquivalentsPeriodIncreaseDecrease: increase/decrease in cash

In [ ]:
cf_pivot_value = load_data("CF", "value")
spark_shape(cf_pivot_value)

(305903, 2909)

In [ ]:
cf_pivot_pd = cf_pivot_value.toPandas()

In [ ]:
cf_pivot_pd_copy = cf_pivot_pd.copy()

### Merge Data from "unknown" statement

In [ ]:
# merge relevant columns from the UN dataset
cf_pivot_pd_copy = prepare_un_values(cf_pivot_pd_copy, [
    'CashAndCashEquivalentsPeriodIncreaseDecrease',
    'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect',
    'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect'
])
cf_pivot_pd_copy.shape

(305903, 2912)

### Cash Increase/Decrease
- 'CashAndCashEquivalentsPeriodIncreaseDecrease',
- 'CashAndCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect',
- 'CashPeriodIncreaseDecrease',
- 'CashPeriodIncreaseDecreaseExcludingExchangeRateEffect',
- 'NetCashProvidedByUsedInContinuingOperations'

In [ ]:
print_null_count(cf_pivot_pd_copy, ['CashAndCashEquivalentsPeriodIncreaseDecrease'])

CashAndCashEquivalentsPeriodIncreaseDecrease   217223


In [ ]:

copy_if_not_empty(cf_pivot_pd_copy, 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashPeriodIncreaseDecrease', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashAndCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'CashPeriodIncreaseDecreaseExcludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInContinuingOperations', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or


# try to find data in joined un data
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashAndCashEquivalentsPeriodIncreaseDecrease', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or
copy_if_not_empty(cf_pivot_pd_copy, 'cpy_CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseExcludingExchangeRateEffect', 
                                    'CashAndCashEquivalentsPeriodIncreaseDecrease') # either or

In [ ]:
print_null_count(cf_pivot_pd_copy, ['CashAndCashEquivalentsPeriodIncreaseDecrease'])

CashAndCashEquivalentsPeriodIncreaseDecrease   188780


### Operation
- NetIncomeLoss
- NetCashProvidedByUsedInOperatingActivities: NetIncome + other positions ergibt diese Position

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetIncomeLoss', 'ProfitLoss', 'NetCashProvidedByUsedInOperatingActivities'])

NetIncomeLoss   195814
ProfitLoss   240347
NetCashProvidedByUsedInOperatingActivities   218636


In [ ]:
# if only ProfitLoss is set, copy content to NetIncomeLoss
# if onlyNetIncomeLoss is set, copy to ProfitLoss
copy_if_not_empty(cf_pivot_pd_copy, 'ProfitLoss', 'NetIncomeLoss')
copy_if_not_empty(cf_pivot_pd_copy, 'NetIncomeLoss', 'ProfitLoss')
copy_if_not_empty(cf_pivot_pd_copy, 'NetIncomeLossAvailableToCommonStockholdersBasic', 'ProfitLoss') # certain CFs just have this position
copy_if_not_empty(cf_pivot_pd_copy, 'NetIncomeLossAvailableToCommonStockholdersBasic', 'NetIncomeLoss')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations', 'NetCashProvidedByUsedInOperatingActivities')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInOperatingActivities', 'ProfitLoss') # certain CFs just have this position
copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInOperatingActivities', 'NetIncomeLoss')

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetIncomeLoss', 'ProfitLoss', 'NetCashProvidedByUsedInOperatingActivities'])

NetIncomeLoss   119998
ProfitLoss   119998
NetCashProvidedByUsedInOperatingActivities   187825


### Investing
- NetCashProvidedByUsedInInvestingActivities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInInvestingActivities'])

NetCashProvidedByUsedInInvestingActivities   219410


In [ ]:
sum_into_empty_target(cf_pivot_pd_copy, 
                      'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations',
                      'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations',
                      'NetCashProvidedByUsedInInvestingActivities')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInInvestingActivitiesContinuingOperations', 'NetCashProvidedByUsedInInvestingActivities')
copy_if_not_empty(cf_pivot_pd_copy, 'CashProvidedByUsedInInvestingActivitiesDiscontinuedOperations', 'NetCashProvidedByUsedInInvestingActivities')

set_to_zero_if_null(cf_pivot_pd_copy, 'NetCashProvidedByUsedInInvestingActivities')

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInInvestingActivities'])

NetCashProvidedByUsedInInvestingActivities   0


### Financing activities
- PaymentsForRepurchaseOfCommonStock: Aktienrückkäufe
- PaymentsOfDividends
- NetCashProvidedByUsedInFinancingActivities

('CashProvidedByUsedInDiscontinuedOperationsFinancingActivities',
 'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations',
 'NetCashProvidedByUsedInFinancingActivities',
 'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations')

#### NetCashProvidedByUsedInFinancingActivities

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInFinancingActivities'])

NetCashProvidedByUsedInFinancingActivities   219518


In [ ]:
sum_into_empty_target(cf_pivot_pd_copy, 
                      'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations',
                      'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations',
                      'NetCashProvidedByUsedInFinancingActivities')

copy_if_not_empty(cf_pivot_pd_copy, 'NetCashProvidedByUsedInFinancingActivitiesContinuingOperations', 'NetCashProvidedByUsedInFinancingActivities')
copy_if_not_empty(cf_pivot_pd_copy, 'CashProvidedByUsedInFinancingActivitiesDiscontinuedOperations', 'NetCashProvidedByUsedInFinancingActivities')

set_to_zero_if_null(cf_pivot_pd_copy, 'NetCashProvidedByUsedInFinancingActivities')

In [ ]:
print_null_count(cf_pivot_pd_copy, ['NetCashProvidedByUsedInFinancingActivities'])

NetCashProvidedByUsedInFinancingActivities   0


#### PaymentsOfDividends
Simply set to 0.0 if no data is present
 'PaymentsOfDividends',
 'PaymentsOfDividendsCommonStock',
 'PaymentsOfDividendsMinorityInterest',
 'PaymentsOfDividendsPreferredStockAndPreferenceStock',
 'PaymentsOfOrdinaryDividends',

In [ ]:
cf_pivot_pd_copy = sum_cols_into_new_target(cf_pivot_pd_copy, 'PaymentsOfDividendsTotal', 
                                 ['PaymentsOfDividends',
                                 'PaymentsOfDividendsCommonStock',
                                 'PaymentsOfDividendsMinorityInterest',
                                 'PaymentsOfDividendsPreferredStockAndPreferenceStock',
                                 'PaymentsOfOrdinaryDividends'])

#### PaymentsForRepurchaseOfCommonStock: Stock buybacks
 'PaymentsForRepurchaseOfCommonStock',
 'PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations',
 'PaymentsForRepurchaseOfConvertiblePreferredStock',
 'PaymentsForRepurchaseOfPreferredStockAndPreferenceStock',
 'PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock',
 'PaymentsForRepurchaseOfRedeemablePreferredStock'

In [ ]:
cf_pivot_pd_copy = sum_cols_into_new_target(cf_pivot_pd_copy, 'PaymentsForRepurchaseOfStockTotal', 
                                 ['PaymentsForRepurchaseOfCommonStock',
                                     'PaymentsForRepurchaseOfCommonStockForEmployeeTaxWithholdingObligations',
                                     'PaymentsForRepurchaseOfConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfPreferredStockAndPreferenceStock',
                                     'PaymentsForRepurchaseOfRedeemableConvertiblePreferredStock',
                                     'PaymentsForRepurchaseOfRedeemablePreferredStock'])

### Save

In [ ]:
cf_pivot_pd_copy[["cik","ticker", "adsh","period","form", "qtrs","fp",
                                     'CashAndCashEquivalentsPeriodIncreaseDecrease',
                                     'NetIncomeLoss', 
                                     'ProfitLoss', 
                                     'NetCashProvidedByUsedInOperatingActivities', 
                                     'NetCashProvidedByUsedInInvestingActivities',
                                     'NetCashProvidedByUsedInFinancingActivities',
                                     'PaymentsOfDividendsTotal',
                                     'PaymentsForRepurchaseOfStockTotal']] \
                  .to_csv(all_processed_folder + "cf_not_cleaned.csv", index=False)